In [1]:
import tifffile
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os
import onlypores as op
from pathlib import Path

In [2]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

In [3]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

In [4]:
database.columns

Index(['Id', 'Original', 'Eq', 'Frontal 90 Right', 'XCT padded', 'Binary',
       'Material Mask', 'Onlypores', 'UT', 'UT aligned', 'UT aligned amp',
       'UT registered', 'UT lente', 'UT aligned lente', 'UT aligned amp lente',
       'UT registered lente', 'Dataset PatchvsVolfrac 3x3',
       'Dataset PatchvsPacth 3x3', 'Dataset PatchvsVolfrac lente 3x3',
       'Dataset PatchvsPacth lente 3x3', 'Dataset PatchvsVolfrac 5x5',
       'Dataset PatchvsPacth 5x5', 'Dataset PatchvsVolfrac lente 5x5',
       'Dataset PatchvsPacth lente 5x5', 'Dataset PatchvsVolfrac 7x7',
       'Dataset PatchvsPacth 7x7', 'Dataset PatchvsVolfrac lente 7x7',
       'Dataset PatchvsPacth lente 7x7'],
      dtype='object')

In [5]:
ids = database['Id']

paths = database['XCT padded']

onlypores_paths = database['Onlypores']

masks = database['Material Mask']

segmenteds = database['Binary']

In [6]:
for i in range(len(ids)):

    if (paths[i] != None) and ((onlypores_paths[i] is None) or (masks[i] is None) or (segmenteds[i] is None)):    

        print(f'Processing {ids[i]}')

        try:

            print(f'Processing {paths[i]}')

            xct_folder = Path(paths[i])
            volume = read_sequence(xct_folder)
            onlypores, sample_mask, binary = op.onlypores(volume)

            output_folder = xct_folder.parent / 'segmentation'

            #save the segmented volume
            write_sequence(output_folder, 'onlypores', onlypores.astype(np.uint8)*255)
            #save the sample mask
            write_sequence(output_folder, 'sample_mask', sample_mask.astype(np.uint8)*255)
            #save the segmented volume
            write_sequence(output_folder, 'binary', binary.astype(np.uint8)*255)

            onlypores_paths[i] = str(output_folder / 'onlypores')
            masks[i] = str(output_folder / 'sample_mask')
            segmenteds[i] = str(output_folder / 'binary')
        
        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)

Processing 1_10
Processing \\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\02_XCT_data\Airbus\Panel Pegaso\Probetas\1\10\padded


Progress:   0%|          | 0/242 [00:00<?, ?it/s]

Progress: 100%|██████████| 242/242 [00:31<00:00,  7.67it/s]


masking
computing otsu
thresholding with value:  131


Saving: 100%|██████████| 242/242 [00:33<00:00,  7.22it/s]


...

Saving complete.


Saving: 100%|██████████| 249/249 [00:37<00:00,  6.65it/s]


Saving complete.


Saving: 100%|██████████| 249/249 [00:37<00:00,  6.64it/s]


Saving complete.


In [7]:
database['Onlypores'] = onlypores_paths
database['Material Mask'] = masks
database['Binary'] = segmenteds

In [8]:
database.to_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv', index=False)